На обучающей выборке, состоящей из отзывов на приложения в интернет-магазине и их оценки (1 - для положительных отзывов, 0 - для отрицательных), требуется научиться прогнозировать тональность отзывов на аналогичные продукты.

С данными можно ознакомиться на сайте: https://www.kaggle.com/c/product-reviews-sentiment-analysis-light

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score

In [2]:
data_train = pd.read_csv('products_sentiment_train.tsv', sep = '\t', names = ['text', 'label']) 
data_test = pd.read_csv('products_sentiment_test.tsv', sep = '\t', usecols = [1]) 
data_train.head()

,text,label
0,"2 . take around 10,000 640x480 pictures .",1
1,i downloaded a trial version of computer assoc...,1
2,the wrt54g plus the hga7t is a perfect solutio...,1
3,i dont especially like how music files are uns...,0
4,i was using the cheapie pail ... and it worked...,1


In [5]:
print 'train data shape is', data_train.shape
print 'test data shape is', data_test.shape
print data_train.label.value_counts()
print 'Positive reviews rate is', 1274./2000

train data shape is (2000, 2)
test data shape is (500, 1)
1    1274
0     726
Name: label, dtype: int64
Positive reviews rate is 0.637


In [6]:
#используем лемматизатор NLTK
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
     def __init__(self):
            self.wnl = WordNetLemmatizer()
     def __call__(self, doc):
         return [self.wnl.lemmatize(t) for t in word_tokenize(doc)]

In [7]:
#составляем pipeline
cvz = CountVectorizer(tokenizer=LemmaTokenizer(), ngram_range = (1,3), analyzer='word')
lr = LogisticRegression()

X_train = data_train.text
y_train = data_train.label

pipeline = Pipeline([('Vectorizer', cvz), 
                     ('Classifier', lr)
                    ])

In [23]:
#проверяем качество модели на кросс валидации, подбираем оптимальные параметры на сетке
from sklearn.grid_search import GridSearchCV

parameters = dict(Vectorizer__ngram_range =[(1,1), (1,2), (1,3)],
                  Vectorizer__analyzer =['word', 'char', 'char_wb'],
                  Classifier__class_weight = [None, 'balanced']
                 )

gs = GridSearchCV(pipeline, param_grid=parameters, cv = 5)
gs.fit(X_train, y_train)

print gs.best_score_, gs.best_params_

0.791 {'Classifier__class_weight': 'balanced', 'Vectorizer__ngram_range': (1, 2), 'Vectorizer__analyzer': 'word'}


In [29]:
#записываем полученную модель в pickle
import pickle

cvz = CountVectorizer(tokenizer=LemmaTokenizer(), ngram_range = (1,2), analyzer='word')
X_vect_train = cvz.fit_transform(X_train)
lr = LogisticRegression(class_weight = 'balanced')
lr.fit(X_vect_train, y_train)

with open('LemmaTokenizer.pkl', 'wb') as f:
    pickle.dump(LemmaTokenizer(),f)
with open('Vectorizer.pkl', 'wb') as f:
    pickle.dump(cvz,f)
with open('LogitR.pkl', 'wb') as f:
    pickle.dump(lr, f)   